## NetLogo interaction through the pyNetLogo connector: sensitivity analysis with SALib

This notebook provides a more advanced example of interaction between NetLogo and a Python environment, using the SALib library (Herman & Usher, 2017; available through the pip package manager) to sample and analyze a suitable experimental design for a Sobol global sensitivity analysis.

In [1]:
# ensuring compliance of code with both python2 and python3

from __future__ import division, print_function
try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import jpype
import pyNetLogo

#Import the sampling and analysis modules for a Sobol variance-based 
#sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol

SALib relies on a problem definition dictionary which contains the number of input parameters to sample, their names (which should here correspond to a NetLogo global variable), and the sampling bounds. 

In [3]:
problem = { 
  'num_vars': 6,
  'names': ['random-seed',
            'grass-regrowth-time',
            'sheep-gain-from-food',
            'wolf-gain-from-food',
            'sheep-reproduce',
            'wolf-reproduce'], 
  'bounds': [[1, 100000],
             [20., 40.], 
             [2., 8.], 
             [16., 32.],
             [2., 8.],
             [2., 8.]]
}

The SALib sampler will automatically generate an appropriate number of samples for Sobol analysis. To calculate first-order, second-order and total sensitivity indices, this gives a sample size of _n*(2p+2)_, where _p_ is the number of input parameters, and _n_ is a baseline sample size which should be large enough to stabilize the estimation of the indices. For this example, we use _n_ = 200 to reduce runtime, for a total of 2800 experiments.

For more complex cases, the EMA Workbench library (Kwakkel, 2017) can be used to parallelize the simulation.

In [4]:
n = 200
#Generates an input array of shape (n*(p+2), p) with rows for each experiment and columns for each input parameter
param_values = saltelli.sample(problem, n, calc_second_order=True)

In [5]:
param_values.shape

(2800, 6)

# running the experiments in parallel using ipyparallel

In [6]:
import ipyparallel

client = ipyparallel.Client()
client.ids

[0, 1, 2, 3]

In [7]:
dview = client[:]
dview.block=True

we need to setup the namespace on the engines. To do this properly we are first pushing the working directory of this notebook to the engines, next we can use an ipyparallel magic to execute code in parallel on each of the engines. Here the code simply involves some imports and a change of the working directory. We first need to change the working directory for otherwise we cannot import pyNetLogo (this is based on the assumption that pyNetLogo is located in the same directory as this notebook, rather than being on the python path). We also need to change the current working directory of the engines to ensure we have the proper path to the file we need to load

note:: there are various solutions to both problems. For example, we could make the netlogo file a keyword argument and pass the absolute path to it. 

In [8]:
import os
dview.push(dict(cwd=os.getcwd()))

[None, None, None, None]

In [10]:
%%px 
import os
os.chdir(cwd)

import jpype
import pyNetLogo
import pandas as pd
import numpy as np


In [11]:
def sim():
    netlogo = pyNetLogo.NetLogoLink(gui=False)
    netlogo.load_model(r'Wolf Sheep Predation_v6.nlogo')

    results = pd.DataFrame(columns=['Avg. sheep', 'Avg. wolves'])
    
    idx, param_values = zip(*samples) #For keeping track of experiment IDs - not really needed
    
    for run in range(len(param_values)):
        #Set the input parameters
        for i, name in enumerate(problem['names']):
            if name == 'random-seed':
                #The NetLogo random seed requires a different syntax
                netlogo.command('random-seed {}'.format(param_values[run][i]))
            else:
                #Otherwise, assume the input parameters are global variables
                netlogo.command('set {0} {1}'.format(name, param_values[run][i]))

        netlogo.command('setup')
        #Run for 100 ticks and return the number of sheep and wolf agents at each time step
        counts = netlogo.repeat_report(['count sheep','count wolves'], 100)    
        #For each run, save the mean value of the agent counts over time
        results.loc[idx[run], 'Avg. sheep'] = counts['count sheep'].values.mean()
        results.loc[idx[run], 'Avg. wolves'] = counts['count wolves'].values.mean()
        
    return results

In [12]:
dview.push(dict(problem=problem))

#Clunky, just to check experiments are properly ordered
# zip is an iterator on python2
dview.scatter('samples',list(zip(range(len(param_values)),param_values))) 

In [13]:
res = dview.apply_sync(sim)

In [14]:
results = pd.concat(res)

# using map and load balanced view

you now use a scatter approach. The more obvious way to achieve the same result would be to use map instead. This also handles for you the experiment ids. 

In [22]:
def sim(experiment):
    netlogo = pyNetLogo.NetLogoLink(gui=False)
    netlogo.load_model(r'Wolf Sheep Predation_v6.nlogo')

    results = pd.DataFrame(columns=['Avg. sheep', 'Avg. wolves'])
    
    #Set the input parameters
    for name, value in zip(problem['names'], experiment):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(value))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, value))

    netlogo.command('setup')
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    counts = netlogo.repeat_report(['count sheep','count wolves'], 100)    
    
    results = pd.Series([counts['count sheep'].values.mean(), 
                         counts['count sheep'].values.mean()], 
                        index=['Avg. sheep', 'Avg. wolves'])        
    return results

In [26]:
lview =  client.load_balanced_view()
results = pd.DataFrame(lview.map_sync(sim, param_values))

In [27]:
results.head()

,Avg. sheep,Avg. wolves
0,125.25,125.25
1,136.85,136.85
2,125.98,125.98
3,136.46,136.46
4,284.34,284.34


In [15]:
results.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2790, 2791, 2792, 2793, 2794, 2795, 2796, 2797, 2798, 2799],
           dtype='int64', length=2800)

In [16]:
results.shape

(2800, 2)

We can then proceed with the analysis, first using a histogram to visualize output distributions for each outcome:

In [ ]:
sns.set_style('white')
sns.set_context('talk')
fig, ax = plt.subplots(1,len(results.columns), sharey=True)

for i, n in enumerate(results.columns):
    ax[i].hist(results[n], 20)
    ax[i].set_xlabel(n)

ax[0].set_ylabel('Counts')

fig.set_size_inches(10,4)
fig.subplots_adjust(wspace=0.1)
#plt.savefig('JASSS figures/SA - Output distribution.pdf', bbox_inches='tight')
#plt.savefig('JASSS figures/SA - Output distribution.png', dpi=300, bbox_inches='tight')
plt.show()

Bivariate scatter plots can be useful to visualize relationships between each input parameter and the outputs. Taking the outcome for the average sheep count as an example, we obtain the following, using the scipy library to calculate the Pearson correlation coefficient (r) for each parameter:

In [ ]:
import scipy

nrow=2
ncol=len(problem['names'])/nrow
fig, ax = plt.subplots(nrow, ncol, sharey=True)
sns.set_context('talk')
y = results['Avg. sheep']

for i, a in enumerate(ax.flatten()):
    x = param_values[:,i]
    sns.regplot(x, y, ax=a, ci=None, color='k',scatter_kws={'alpha':0.2, 's':8, 'color':'gray'})
    pearson = scipy.stats.pearsonr(x, y)
    a.annotate("r: {:6.3f}".format(pearson[0]), xy=(0.15, 0.85), xycoords='axes fraction',fontsize=13)
    if divmod(i,ncol)[1]>0:
        a.get_yaxis().set_visible(False)
    a.set_xlabel(problem['names'][i])
    a.set_ylim([0,1.1*np.max(y)])

fig.set_size_inches(9,9,forward=True) 
fig.subplots_adjust(wspace=0.2, hspace=0.3)
#plt.savefig('JASSS figures/SA - Scatter.pdf', bbox_inches='tight')
#plt.savefig('JASSS figures/SA - Scatter.png', dpi=300, bbox_inches='tight')
plt.show()

This indicates a positive relationship between the "sheep-gain-from-food" parameter and the mean sheep count, and negative relationships for the "wolf-gain-from-food" and "wolf-reproduce" parameters.

We can then use SALib to calculate first-order (S1), second-order (S2) and total (ST) Sobol indices, to estimate each input's contribution to output variance. By default, 95% confidence intervals are estimated for each index.

In [ ]:
Si = sobol.analyze(problem, results['Avg. sheep'].values, calc_second_order=True, print_to_console=False)

As a simple example, we first select and visualize the first-order and total indices for each input, converting the dictionary returned by SALib to a dataframe.

In [ ]:
Si_filter = {k:Si[k] for k in ['ST','ST_conf','S1','S1_conf']}
Si_df = pd.DataFrame(Si_filter, index=problem['names'])

In [ ]:
Si_df

In [ ]:
sns.set_style('white')
fig, ax = plt.subplots(1)
Si_df.plot(kind='bar', ax=ax)
fig.set_size_inches(8,4)

#plt.savefig('JASSS figures/SA - Indices.pdf', bbox_inches='tight')
#plt.savefig('JASSS figures/SA - Indices.png', dpi=300, bbox_inches='tight')

plt.show()

The "sheep-gain-from-food" parameter has the highest ST index, indicating that it contributes over 50% of output variance when accounting for interactions with other parameters. However, it can be noted that the confidence bounds are overly broad due to the small _n_ value used for sampling, so that a larger sample would be required for reliable results. For instance, the S1 index is estimated to be larger than ST for the "random-seed" parameter, which is an artifact of the small sample size.

We can use a more sophisticated visualization to include the second-order interactions between inputs.

In [ ]:
import itertools
from math import pi


def normalize(x, xmin, xmax):
    return (x-xmin)/(xmax-xmin)


def plot_circles(ax, locs, names, max_s, stats, smax, smin, fc, ec, lw, 
                 zorder):
    s = np.asarray([stats[name] for name in names])
    s = 0.01 + max_s * np.sqrt(normalize(s, smin, smax))
    
    fill = True
    for loc, name, si in zip(locs, names, s):
        if fc=='w':
            fill=False
        else:
            ec='none'
        
        circle = plt.Circle((loc, 1), radius=si, ec=ec, fc=fc,
                            zorder=zorder, lw=lw, fill=True)
        ax.add_patch(circle)
        

def filter(sobol_indices, names, locs, criterion, threshold):
    if criterion in ['ST', 'S1', 'S2']:
        data = sobol_indices[criterion]
        data = np.abs(data)
        data = data.flatten() # flatten in case of S2
        # TODO:: remove nans
        
        filtered = ([(name, locs[i]) for i, name in enumerate(names) if 
                     data[i]>threshold])
        filtered_names, filtered_locs = zip(*filtered)
    elif criterion in ['ST_conf', 'S1_conf', 'S2_conf']:
        raise NotImplementedError
    else:
        raise ValueError('unknown value for criterion')

    return filtered_names, filtered_locs


def plot_sobol_indices(sobol_indices, criterion='ST', threshold=0.01):
    '''plot sobol indices on a radial plot
    
    Parameters
    ----------
    sobol_indices : dict
                    the return from SAlib
    criterion : {'ST', 'S1', 'S2', 'ST_conf', 'S1_conf', 'S2_conf'}, optional
    threshold : float
                only visualize variables with criterion larger than cutoff
             
    '''
    max_linewidth_s2 = 10
    max_s_radius = 0.1
    
    # prepare data
    # use the absolute values of all the indices
    sobol_indices = {key:np.abs(stats) for key, stats in sobol_indices.items()}
    
    # dataframe with ST and S1
    sobol_stats = {key:sobol_indices[key] for key in ['ST', 'S1']}
    sobol_stats = pd.DataFrame(sobol_stats, index=problem['names'])

    smax = sobol_stats.max().max()
    smin = sobol_stats.min().min()

    # dataframe with s2
    s2 = pd.DataFrame(sobol_indices['S2'], index=problem['names'], 
                      columns=problem['names'])
    s2max = s2.max().max()
    s2min = s2.min().min()

    names = problem['names']
    n = len(names)
    ticklocs = np.linspace(0, 2*pi, n+1)
    locs = ticklocs[0:-1]

    filtered_names, filtered_locs = filter(sobol_indices, names, locs,
                                           criterion, threshold)
    
    # setup figure
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='polar')
    ax.grid(False)
    ax.spines['polar'].set_visible(False)
    ax.set_xticks(ticklocs)
    ax.set_xticklabels(names)
    ax.set_yticklabels([])
    ax.set_ylim(ymax=1.4)
    legend(ax)

    # plot ST
    plot_circles(ax, filtered_locs, filtered_names, max_s_radius, 
                 sobol_stats['ST'], smax, smin, 'w', 'k', 1, 9)

    # plot S1
    plot_circles(ax, filtered_locs, filtered_names, max_s_radius, 
                 sobol_stats['S1'], smax, smin, 'k', 'k', 1, 10)

    # plot S2
    for name1, name2 in itertools.combinations(zip(filtered_names, filtered_locs), 2):
        name1, loc1 = name1
        name2, loc2 = name2

        weight = s2.ix[name1, name2]
        lw = 0.5+max_linewidth_s2*normalize(weight, s2min, s2max)
        ax.plot([loc1, loc2], [1,1], c='darkgray', lw=lw, zorder=1)

    return fig


from matplotlib.legend_handler import HandlerPatch
class HandlerCircle(HandlerPatch):
    def create_artists(self, legend, orig_handle,
                       xdescent, ydescent, width, height, fontsize, trans):
        center = 0.5 * width - 0.5 * xdescent, 0.5 * height - 0.5 * ydescent
        p = plt.Circle(xy=center, radius=orig_handle.radius)
        self.update_prop(p, orig_handle, legend)
        p.set_transform(trans)
        return [p]

def legend(ax):
    some_identifiers = [plt.Circle((0,0), radius=5, color='k', fill=False, lw=1),
                        plt.Circle((0,0), radius=5, color='k', fill=True),
                        plt.Line2D([0,0.5], [0,0.5], lw=8, color='darkgray')]
    ax.legend(some_identifiers, ['ST', 'S1', 'S2'],
              loc=(1,0.75), borderaxespad=0.1, mode='expand',
              handler_map={plt.Circle: HandlerCircle()})


sns.set_style('whitegrid')
fig = plot_sobol_indices(Si, criterion='ST', threshold=0.005)
fig.set_size_inches(5,5)
#plt.savefig('JASSS figures/SA - Interactions.pdf', bbox_inches='tight')
#plt.savefig('JASSS figures/SA - Interactions.png', dpi=300, bbox_inches='tight')
plt.show()

In this case, the sheep-gain-from-food variable has strong interactions with the wolf-gain-from-food and sheep-reproduce inputs in particular. The size of the ST and S1 circles correspond to the normalized variable importances.

Finally, the kill_workspace() function shuts down the NetLogo instance.

In [ ]:
netlogo.kill_workspace()